### **Señas Chapinas: Traductor de LENSEGUA**
#### *Módulo de Procesamiento de Lenguaje Natural*

Stefano Alberto Aragoni Maldonado

-------------------

#### **Generación de Interpretaciones**

En este programa se busca poder interpretar las 200 frases del conjunto de validación. En realidad, se generarán las interpretaciones utilizando dos modelos diferentes; el modelo GPT-3.5-Turbo base y el modelo GPT-3.5-Turbo fine-tuneado para la tarea de interpretación de LENSEGUA. 

Además, por cada modelo se generarán cinco posibles interpretaciones para cada frase. Esto debido a que cada interpretación utilizará un prompt diferente.

Por lo tanto, ya que son 200 frases, 2 modelos, y 5 prompts, se generarán 2,000 interpretaciones en total.

___

#### *Importar librerías*
Como primer paso, se importan las librerías necesarias para el desarrollo del problema.

In [14]:
import numpy as np
import pandas as pd
import csv
import re
from openai import OpenAI

___

#### *Cargar el dataset*

Luego, se carga el dataset que contiene las frases en español y su contraparte en LENSEGUA.

In [15]:
data = pd.read_csv("../../dataset/processed/validation_data.csv")   # Load the dataset

data.head()

,LENSEGUA,ESPAÑOL
0,Hoy estudiantes ocupados mucho,Hoy los estudiantes están muy ocupados.
1,Tu trabajar dónde pregunta,¿Dónde trabajas?
2,Ojalá hoy carro mucho no,Espero que hoy no haya tráfico.
3,Haber carro mucho,Hay tráfico.
4,Pasado yo tarde llegar porque carro mucho,Llegué tarde porque había tráfico.


___

#### *Definición de prompts*

Posteriormente, se definen los prompts que se utilizarán para generar las interpretaciones de las frases en LENSEGUA.

In [10]:
prompt_v0 = 'Eres un intérprete experto en Lengua de Señas de Guatemala (LENSEGUA). El usuario te proporcionará frases en español escritas utilizando la gramática de LENSEGUA. Tu tarea es interpretarlas y convertirlas en español gramaticalmente correcto.'

prompt_v1 = 'Soy un intérprete experto en Lengua de Señas de Guatemala (LENSEGUA). El usuario me proporcionará frases en español escritas utilizando la gramática de LENSEGUA. Mi tarea es interpretarlas y convertirlas en español gramaticalmente correcto.'

prompt_v2 = 'Eres un intérprete experto en Lengua de Señas de Guatemala (LENSEGUA). El usuario te proporcionará frases en español escritas utilizando la gramática de LENSEGUA. Tu tarea es interpretarlas y convertirlas en español gramaticalmente correcto, teniendo en cuenta la siguiente regla específica: Las palabras “pasado”, “futuro” y “antes” indican la conjugación verbal. Debes conjugar correctamente los verbos en tu respuesta sin incluir las palabras “pasado”, “futuro” o “antes”.'

prompt_v3 = 'Eres un intérprete experto en Lengua de Señas de Guatemala (LENSEGUA). El usuario te proporcionará frases en español escritas utilizando la gramática de LENSEGUA. Tu tarea es interpretarlas y convertirlas en español gramaticalmente correcto, teniendo en cuenta las siguientes reglas específicas: 1. No debes interpretar las frases como preguntas, ni agregar una entonación interrogativa, a menos que incluyan las palabras "pregunta" o "sí o no". 2. Las palabras “pasado”, “futuro” y “antes” indican la conjugación verbal. Debes conjugar correctamente los verbos en tu respuesta sin incluir las palabras “pasado”, “futuro” o “antes”. 3. La frase “carro mucho” debe interpretarse como “tráfico”. 4. La frase “bien mucho” debe interpretarse como “muy bien”.'

prompt_v4 = 'Eres un intérprete experto en Lengua de Señas de Guatemala (LENSEGUA). El usuario te proporcionará frases en español escritas utilizando la gramática de LENSEGUA. Tu tarea es interpretarlas y convertirlas en español gramaticalmente correcto, teniendo en cuenta las siguientes reglas específicas: 1. No debes interpretar las frases como preguntas, ni agregar una entonación interrogativa, a menos que incluyan las palabras "pregunta" o "sí o no". 2. Las palabras “pasado”, “futuro” y “antes” indican la conjugación verbal. Debes conjugar correctamente los verbos en tu respuesta sin incluir las palabras “pasado”, “futuro” o “antes”. 3. La frase “carro mucho” debe interpretarse como “tráfico”. 4. La frase “bien mucho” debe interpretarse como “muy bien”. 5. Mantén el orden de los elementos de la interpretación final lo más similar posible al texto original cuando sea gramaticalmente válido en español. Por ejemplo, si la frase en LENSEGUA es "Por favor cuando tu salir luz apagar", la interpretación debe ser "Por favor cuando salgas apaga la luz" en lugar de "Por favor apaga la luz cuando salgas".'

___

#### *Definición de modelos*

Asimismo, se definen los modelos que se utilizarán para generar las interpretaciones de las frases en LENSEGUA.

In [11]:
model_finetuned = "ft:gpt-3.5-turbo-0125:personal:lensegua-16-02-2:9wdLYlLV:ckpt-step-262"

model_original = "gpt-3.5-turbo-0125"

___

#### *Función para generar interpretaciones*

Con todo lo anterior, se define una función que permitirá generar las interpretaciones de las frases en LENSEGUA. Esta recibe como parámetros el dataset, el prompt y el modelo a utilizar.

Como resultado, se obtiene un archivo CSV que contiene la información del dataset original y las interpretaciones generadas. 

In [ ]:
def get_responses(data, prompt, model, file_name, temp=1):

    # OpenAI API client gets initialized with the API key
    client = OpenAI(
        api_key=''
    )

    # Rows for new dataset
    rows = []

    for i, row in data.iterrows():

        # Get rows from the dataset
        spanish = row["ESPAÑOL"]
        lensegua = row["LENSEGUA"]

        # Generate the response
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": lensegua + " ->"}
            ],
            max_tokens=150,
            temperature=temp,
        )

        # Get the response from the API
        response = response.choices[0].message.content

        # Remove " from the response
        response = response.replace('"', '').replace('“', '').replace('”', '').replace("'", '').replace(",", "").replace("*", "").replace("\n", "")

        # Append the rows to the new dataset
        rows.append([spanish, lensegua, response])

    # Create the new dataset
    new_data = pd.DataFrame(rows, columns=["ESPAÑOL", "LENSEGUA", "RESPUESTA"])

    # Save the new dataset
    new_data.to_csv(file_name, index=False)

In [6]:
# Get responses for the original model
get_responses(data, prompt_v0, model_original, "../../dataset/interpretations/original_pv0.csv")

get_responses(data, prompt_v1, model_original, "../../dataset/interpretations/original_pv1.csv")

get_responses(data, prompt_v2, model_original, "../../dataset/interpretations/original_pv2.csv")

get_responses(data, prompt_v3, model_original, "../../dataset/interpretations/original_pv3.csv")

get_responses(data, prompt_v4, model_original, "../../dataset/interpretations/original_pv4.csv")

In [7]:
# Get responses for the fine-tuned model
get_responses(data, prompt_v0, model_finetuned, "../../dataset/interpretations/finetuned_pv0.csv")

get_responses(data, prompt_v1, model_finetuned, "../../dataset/interpretations/finetuned_pv1.csv")

get_responses(data, prompt_v2, model_finetuned, "../../dataset/interpretations/finetuned_pv2.csv")

get_responses(data, prompt_v3, model_finetuned, "../../dataset/interpretations/finetuned_pv3.csv")

get_responses(data, prompt_v4, model_finetuned, "../../dataset/interpretations/finetuned_pv4.csv")